In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import shutil
from time import sleep
import pymorphy2
from lru import LRU

In [2]:
morph = pymorphy2.MorphAnalyzer()
PYMORPHY_CACHE = LRU(10000000)
def pymorphy_tokenizer(text):
    for word in text:
        word_hash = hash(word)
        if (word_hash not in PYMORPHY_CACHE):
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [5]:
query = all_groups['QueryId'].values
doc = all_groups['DocumentId'].values
q_to_doc = dict()
for i in tqdm(range(query.shape[0])):
    if query[i] not in q_to_doc:
        q_to_doc[query[i]] = []
    q_to_doc[query[i]].append(doc[i])

100%|██████████| 606049/606049 [00:00<00:00, 1113958.32it/s]


In [7]:
os.makedirs('clean_doc_collection')

In [8]:
q_ind = all_groups['QueryId'].unique()
for i in tqdm(q_ind):
    os.makedirs('clean_doc_collection/'+str(i))

100%|██████████| 6311/6311 [00:02<00:00, 2563.55it/s]


In [9]:
#читаем данные из gzip файла
with gzip.open("docs.tsv.gz") as file:
    for line in tqdm(file):
        a,b,c = line.decode('utf-8', errors = 'ignore').lower().strip('\n').split('\t')
        c = c.split()
        c = [word for word in pymorphy_tokenizer(c)]
        c = ' '.join(c)
        text = c.encode('utf-8')
        if int(a) in doc_to_q:
            #тексты
            try:
                with gzip.open(a+'.gz', 'wb') as f:
                    f.write(text)
            except OSError:
                sleep(3)
                with gzip.open(a+'.gz', 'wb') as f:
                    f.write(text)
            q_list = doc_to_q[int(a)]
            for elem in q_list:
                shutil.copy2(a+'.gz', 'clean_doc_collection/'+str(elem)+'/'+a+'.gz')
            try:
                os.remove(a+'.gz')
            except PermissionError:
                sleep(3)
                os.remove(a+'.gz')

582167it [9:30:24, 17.01it/s] 
